In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch

from mvn.utils.minimon import MiniMon
from mvn.utils.misc import flush_cache

from mvn.ipynb import get_args, get_config, build_labels
from train import setup_dataloaders, setup_experiment, build_env, do_train

In [3]:
flush_cache()

is_distributed = False
master = True
device = torch.device(0)
print('using dev {}'.format(device))
    
args = get_args()

# just to get a feeling of the dataset
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10000)
# labels, mask, indices = build_labels(config.dataset.train.labels_path, 10, allowed_subjects=['S9', 'S11'])

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [20]:
config = get_config(args)
do_train(None, None, config, device, is_distributed, master)

successfully loaded pretrained weights from /home/stefano/Scuola/tud/_classes/thesis/milestones/06.05_12.05_solving_the_MLP_problem/weights_cam2cam_model.pth
cam2cam model:
                roto_extractor has      25606 params (~ 50.2) %
               trans_extractor has      25411 params (~ 49.8) %
total params:                51017
  training dataset length: 12
  validation dataset length: 5
Experiment name: human36m_alg_AlgebraicTriangulationNet@07.05.2021-11:12:24
#[do_train]: cam2cam loss weights: geo: 0.0, trans: 0.0, proj: 1.0
#[do_train]: epoch    0 has started!
#[cam2cam]: I'm using GT data
------------- R
R GT 0 0
[[-0.7000174   0.11276358 -0.70516676]
 [-0.17936999  0.9280348   0.32646272]
 [ 0.6912323   0.35501534 -0.62941396]]
[[ 1.0000001e+00 -7.6752560e-09 -4.2079044e-08]
 [-7.6752560e-09  1.0000000e+00  5.6179726e-08]
 [-4.2079044e-08  5.6179726e-08  9.9999994e-01]]
R PRED 0 0
[[-0.42134055 -0.02296608  0.9066117 ]
 [ 0.8974549   0.1333616   0.42046332]
 [-0.13056357  0

ZeroDivisionError: division by zero